In [ ]:
%config IPCompleter.greedy=True

In [ ]:
imp_urls=['https://www1.nseindia.com/homepage/Indices1.json','https://www1.nseindia.com/live_market/dynaContent/live_analysis/gainers/niftyGainers1.json'
          ,'https://archives.nseindia.com/content/equities/series_change.csv'
   
     ]

In [ ]:
import io
import os
import logging
import requests
import math

from urllib.parse import quote

import traceback
import time, sched
from datetime import timedelta,datetime
from pathlib import Path

from tabulate import tabulate 
from IPython.display import display,HTML

import pandas as pd
import numpy as np
from pandas import json_normalize  

import pdfkit
from PyPDF2 import PdfFileReader, PdfFileWriter
import tempfile
import base64
from html import unescape,escape

from matplotlib.ticker import PercentFormatter 
import matplotlib.pyplot as plt
from matplotlib import cm,colors

import seaborn as sns

import cufflinks as cf
import plotly.offline as py
import plotly.graph_objs as go

logging.basicConfig(level=logging.INFO)

nse_base_url='https://www.nseindia.com/'
nse1_base_url='https://www1.nseindia.com/'

ticker_url='https://www.nseindia.com/api/quote-equity'
ticker_param = {'symbol':'TANLA'}

chrome_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Whale/0.10.36.11 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    # 'Host': 'www.matchesfashion.com',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0',
    'Accept-Encoding':'gzip, deflate'}


my_initial_port = {}
source_symb = 'SYMBOL'
target_symb = 'i_symbol'
closing_price_keys= ['pr_close','pr_lastPrice']

default_interested_columns = [target_symb,'pr_pChange','pr_lastPrice','pr_open','pr_previousClose'
                      ,'pr_close','m_series','pr_pBand','i_companyName','m_industry','request_time']

base_cookies = {}

In [ ]:
# Show up to 15 cols, 50 rows by default
#pd.set_option('display.max_cols', 15)
#pd.set_option('display.max_rows', 50)
# Suitable default display for floats

pd.options.display.float_format = '{:,.2f}'.format

np.set_printoptions(formatter={'float_kind':"{:.3f}".format})
cf.go_offline()
%matplotlib inline
plt.rcParams['figure.figsize'] = (6, 6)
# This one is optional -- change graphs to SVG
# Only use if you don't have a lot of points/lines
# on your graph.
# Can also just use ['retina'] if you don't want SVG.
%config InlineBackend.figure_formats = ['retina', 'svg']

default_layout = cf.Layout(
    height=600,
    width=600
)

#Adding wkhtmltox to path for pdf gen
os.environ["PATH"] = os.environ["PATH"] + os.pathsep + 'path\\wkhtmltox\\bin'

#

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Aggregation Function , base metrics


def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = str(n)+'%'
    return percentile_

def last(x):
    return x.tail(1)

def first(x):
    return x.head(1)

def p_change(x):
    return (x.tail(1).values[0]-x.head(1).values[0])*100/x.head(1).values[0]

concat = lambda x: ' '.join(x.unique())

time_dims = { 
    '1 week' : lambda x: x['time'] > datetime.today() - timedelta(days=7),
    '1 month' : lambda x: x['time'] > datetime.today() - timedelta(days=30),
    #'3 month' : lambda x: x['time'] > datetime.today() - timedelta(days=90),
    '6 month' :  lambda x: x['time'] > datetime.today() - timedelta(days=180),
    '1 year' : lambda x: x['time'] > datetime.today() - timedelta(days=365),
    '5 year' : lambda x: x['time'] > datetime.today() - timedelta(days=365*5),
    #'Overall' : lambda x: True,
            }

metrics =[np.mean, np.std, 'min', 'max',first,last, percentile(25),percentile(50), percentile(75), percentile(95),p_change]

In [ ]:
class MyURL:
    def __init__(self, name, url,content_type='application/json',node="data",data=None,parent_page=nse_base_url,payload={}):
        self.name = name
        self.url = url
        self.content_type=content_type
        self.node=node
        self.data = data
        self.parent_page = parent_page
        self.payload = payload
    def set_data(data):
        self.data=data
    def info():
        print(str(self))
        display(data)
    def __str__(self):
        return " ".join([self.name,self.url,str(self.payload)]);
        


In [ ]:
key_map = {'priceInfo':'pr_','info':'i_','metadata':'m_','pr_pPriceBand':'pr_pBand_'}
# Flatten json
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], get_prefix(name + str(a) ))
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

def get_prefix(key):
    return key_map.get(key,key+'_')

def display_df_html(df):
    display(HTML(df.to_html()))

def display_full(df):
    with pd.option_context("display.max_rows", df.shape[0],'display.max_columns', df.shape[1]):
        display(df)

In [ ]:
# HTTP based function

def loadURL(myurl,latest):
    print('loading url ', myurl)
    response=requests.get(myurl.url,params=myurl.payload,cookies=get_cookies(myurl.parent_page),headers=chrome_headers,timeout=20)
    print(str(myurl),response)
    s = response.text
    try:
        if(myurl.content_type=='application/json'):
            myurl.data = json_normalize(pd.read_json(io.StringIO(s))[myurl.node])
        elif(myurl.content_type=='json'):
            myurl.data = response.json()
        elif(myurl.content_type=='csv'):
            myurl.data = pd.read_csv(io.StringIO(s))
        elif(myurl.content_type=='html'):
            myurl.data = pd.read_html(s)
        else:
            myurl.data=s
        return myurl.data
    except Exception:
        print('Error parsing data ',s)

def get_cookies(parent_url=nse_base_url):
    global base_cookies
    if(parent_url not in base_cookies):
        print('loading cookies for',parent_url)
        base_cookies[parent_url] = requests.get(parent_url,headers=chrome_headers)
        display(base_cookies)

    return base_cookies[parent_url].cookies
    
def load_ticker_info(ticker):
    global base_cookies
    print('loading ticker info',ticker_url,ticker)
    payload = {'symbol':ticker}
    ticker_data = None
    try:
        r = requests.get(ticker_url,params=payload,headers=chrome_headers,cookies=get_cookies())
        ticker_data = flatten_json(r.json());
        ticker_data['request_time'] = datetime.today()
    except Exception:
        traceback.print_exc()
        base_cookies = {} 
    
    return ticker_data

def load_index_data(index_name):
    url = 'https://www.nseindia.com/api/equity-stockIndices?csv=false&index='+quote(index_name)
    data_url = MyURL(index_name,url,content_type='data',parent_page=nse_base_url)
    text = loadURL(data_url,False)
    #print(text)
    text = data_url.data.replace(' \n','',13)
    nifty_next_50 = pd.read_csv(io.StringIO(text[text.find('"'):]))  #[['SYMBOL']]
    nifty_next_50 = nifty_next_50.iloc[1:]
    return nifty_next_50

def load_ca_info(days=365):
    #base_url = 'https://api.moneycontrol.com/mcapi/v1/stock/corporate-action?deviceType=W&scId=TCS&section=d&start=0&limit=10'
    #nse_dividend_url='https://www1.nseindia.com/corporates/datafiles/CA_TCS_LAST_12_MONTHS.csv'
    nse_dividend_all ='https://www.nseindia.com/api/corporates-corporateActions'
    to_time = datetime.today().strftime("%d-%m-%Y")
    from_time = (datetime.today() - timedelta(days=days)).strftime("%d-%m-%Y")
    payload = {'index':'equities','csv':'true','from_date':from_time,'to_date':to_time}
    data_url = MyURL('Dividend',nse_dividend_all,payload=payload,content_type='data',parent_page=nse_base_url)
    text = loadURL(data_url,False)
    data_frame = pd.read_csv(io.StringIO(text[text.find('"'):]))
    data_frame.rename(columns={"Symbol": "SYMBOL"},inplace=True)
    return data_frame

In [ ]:
# dataframe filter function
def time_filter(target_df,time_key):
    return target_df[target_df.apply(time_dims[time_key],axis=1)].reset_index()

def ticker_filter(target_df,ticker):
    return target_df[target_df['SYMBOL']==ticker].reset_index()

def apply_filter(data,column,value):
    return data[data[column]==value].reset_index()

def order_time_dims(data,time_dim_column):
    data[time_dim_column] = pd.Categorical(data[time_dim_column],ordered=True,categories=time_dims.keys())
    return data
def filter_by_index(df,column='index',values=[]):
    return df[df.index.get_level_values(column).isin(values)]

In [ ]:


buy_tax_overhead = 0.0
sell_tax_overhead = buy_tax_overhead

def __count_days(x):
    if (pd.isnull(x[1])):
        return (datetime.today().date()-x[0]).days
    return (x[1] -x[0]).days

# File based portfolio load/update function
def load_portfolio_file(file,populate_avg=True):
    if(os.path.isfile('output/'+file)):
        file_df = pd.read_csv ('output/'+file)
    else:
        file_df = pd.read_csv (file)

    if (populate_avg and 'AVG' not in file_df.columns):
        file_df['AVG'] = file_df['INVESTMENT']/file_df['QTY'] 
    
    if("DATE" in file_df.columns):
        file_df['DATE'] =  pd.to_datetime(file_df['DATE']).dt.date
        file_df['SELL_DATE'] =  pd.to_datetime(file_df['SELL_DATE']).dt.date
        file_df['DAYS'] = file_df[['DATE','SELL_DATE']].apply(__count_days,axis=1)
        file_df['CATEGORY'] = file_df['DAYS'].apply(trade_time_category)
    
    return file_df
def get_today_dir():
    today_dir = "output/"+datetime.today().strftime('%Y%m%d')+"/"
    Path(today_dir).mkdir(parents=True, exist_ok=True)
    return today_dir
def update_portfolio_file(portfolio_df,file,update_current_port=True):
    
    portfolio_df.to_csv(get_today_dir()+file,index=False)
    if(update_current_port):
        portfolio_df[["SYMBOL","DATE","INVESTMENT","REAL","QTY","SELL_DATE"]].to_csv('output/'+file,index=False)


def buy_position(file,symbol,qty,avg,date):
    port_data = load_portfolio_file(file)
    invst = (qty * avg) + buy_tax_overhead
    port_data = port_data.append({'DATE':date,'SYMBOL':symbol,'REAL':0,'QTY':qty,'INVESTMENT':invst,'SELL_DATE':np.nan}, ignore_index=True)
    update_portfolio_file(port_data,file,True)
    return port_data
   
def __update_sell_position(ticker_record,computed_qty,avg,date):
    if (ticker_record['QTY'] <=0) :
        return ticker_record
    
    if (ticker_record['QTY'] - computed_qty > 0):
        print('This need to be handled - split record')
        investment_rem =  ticker_record['INVESTMENT'] - (ticker_record['AVG'] * computed_qty)
        return ticker_record
    
    amt = computed_qty * avg - sell_tax_overhead
    ticker_record['QTY'] =  ticker_record['QTY'] - computed_qty
    ticker_record['REAL'] = ticker_record['REAL'] + amt - (ticker_record['AVG'] * computed_qty)
    ticker_record['SELL_DATE'] = date
    
    
    
    return ticker_record

def sell_position(portfolio_name,ticker,qty,avg,date):
    port_data = load_portfolio_file(portfolio_name)
     
    ticker_records = ticker_filter(port_data, ticker).drop('index',axis=1).sort_values(['DATE'],ascending=True)
    display(ticker_records)
    updated_ticker_records = pd.DataFrame({})
    remaining_qty = qty
    for index, row in ticker_records.iterrows():
        computed_qty = min(row['QTY'],remaining_qty)
        remaining_qty = remaining_qty - computed_qty
        updated_ticker_records = updated_ticker_records.append(__update_sell_position(row,computed_qty,avg,date))
    
    #ticker_record['DATE'] = date
    port_data = port_data[port_data['SYMBOL']!= ticker].append(updated_ticker_records)
        
    update_portfolio_file(port_data,portfolio_name)
    return port_data


In [ ]:
def trade_time_category(days):
    if (days < 60):
        return '< 2 Month'
    elif (days < 366):
        return '< 1 Year'
    elif (days < 1830):
        return '> 1 Year'
    else:
        return '> 5 Year'


default_content_type='text/plain'

def output_to_pdf(cell_results,file_name):
    
    final_pdf_string = ''
    for cell_result in cell_results:
        for output in cell_result.outputs:
            final_pdf_string = final_pdf_string + get_pdf_output_string(output)
        final_pdf_string = final_pdf_string + '<br><div style="page-break-before:always">&nbsp;</div>'

    print('writing to file now')        
    pdfkit.from_string(final_pdf_string, get_today_dir() + file_name)
    
def get_pdf_output_string(output):
    is_not_printed = True
    for content_type in output.data:
        if ('text/html' == content_type):
            return output.data[content_type]
            is_not_printed = False
        elif ('image/svg+xml' == content_type):
            return '<img src="data:image/svg+xml,'+ escape(output.data[content_type]) +'"> </img>'
            is_not_printed = False
    
    if(is_not_printed and (len(output.data)>1 or default_content_type not in output.data )):
        print('None of the content_type handling, so using  for now ',output.data.keys())
    
    if(is_not_printed and default_content_type in output.data):        
        return '<pre>'+output.data[default_content_type] +'</pre>'

    return ""



breaker_text = '<br>'
def add_page_break():
    display(HTML(breaker_text))


def create_pdf(content,file_name):
    print ('new file creation',file_name)
    pdfkit.from_string(content, file_name)
    return file_name

def pdf_concat(input_files, target_file):
    input_streams = []
    try:
        # First open all the files, then produce the output file, and
        # finally close the input files. This is necessary because
        # the data isn't read from the input files until the write
        # operation. Thanks to
        # https://stackoverflow.com/questions/6773631/problem-with-closing-python-pypdf-writing-getting-a-valueerror-i-o-operation/6773733#6773733
        display('files to concat',input_files)
        with open(target_file,'wb') as file:
            for input_file in input_files:
                input_streams.append(open(input_file, 'rb'))
            writer = PdfFileWriter()
            for reader in map(PdfFileReader, input_streams):
                for n in range(reader.getNumPages()):
                    writer.addPage(reader.getPage(n))
            writer.write(file)
    finally:
        for f in input_streams:
            f.close()
            
def write_output_to_pdf(cell_results, final_file_name):    
    tmp_dir = tempfile.TemporaryDirectory(dir = "output/")
    try:
        today_dir = "output/"+datetime.today().strftime('%Y%m%d')+"/"
        default_content_type='text/plain'
        final_pdf_string = ''
        file_names = []
        file_name = tmp_dir.name + '/tmp_pdf_'+str(len(file_names)) + '.pdf'
        for cell_result in cell_results:
            for output in cell_result.outputs:
                current_string = get_pdf_output_string(output)
                if(current_string == breaker_text):
                    file_names.append(create_pdf(final_pdf_string, file_name))
                    file_name = tmp_dir.name + '/tmp_pdf_'+str(len(file_names)) + '.pdf'
                    final_pdf_string = ''
                else:
                    final_pdf_string = final_pdf_string + current_string

        file_names.append(create_pdf(final_pdf_string, file_name))     
        
        pdf_concat(file_names,get_today_dir() + final_file_name )
    finally:
        tmp_dir.cleanup()

In [ ]:

nse_cached_data = {}

def get_latest_data(tickers,latest):
    global nse_cached_data
    is_market_latest = True
    
    if(len(nse_cached_data)!=0):
        load_timing = pd.DataFrame(list(nse_cached_data.values()))['request_time'].agg(['min','max'])
    else:
        load_timing = []
    
    # do not refresh data for sat and sun
    if(datetime.today().weekday() > 4 or latest == False or (len(load_timing) > 0 and load_timing[0].hour >= 16)):
        is_market_latest = False

    if(datetime.today().hour >= 18 and len(load_timing) > 0 and load_timing[0].hour < 18):
        is_market_latest = True
        
    available_cached_data = {}
    for symb in tickers:
        if (is_market_latest or symb not in nse_cached_data):
            ticker_data = load_ticker_info(symb)
            if(ticker_data is not None):
                nse_cached_data[symb] = ticker_data
            
        if(symb in nse_cached_data):
            available_cached_data[symb]= nse_cached_data[symb]
    
    return available_cached_data.values()
    
def populate_nse_data(my_port_df,latest=True,interested_columns=default_interested_columns):
    nse_current_data = get_latest_data(my_port_df['SYMBOL'].unique(),latest)
    new_df = pd.DataFrame(nse_current_data,columns=interested_columns)
    new_df = new_df.rename(columns={target_symb: source_symb})
    my_port_value = my_port_df.merge(new_df)
    return my_port_value

def enrich_portfolio(my_port_value):
    start_time = datetime.today()
    price_key=closing_price_keys[1]
    if (my_port_value[closing_price_keys[0]].unique()[0]!=0):
        price_key = closing_price_keys[0]
    display(HTML('<b>Share Price Key: </b>'+price_key))
    today_val_formule = 'QTY * '+ price_key
    my_port_value.insert(3, 'MKT_VALUE', my_port_value.eval(today_val_formule))
    my_port_value.insert(4, 'CHANGE', my_port_value.eval('MKT_VALUE - QTY * pr_previousClose'))
    my_port_value.insert(5, 'UNREAL', my_port_value.eval('MKT_VALUE-INVESTMENT'))
    my_port_value.insert(6, 'UNREAL %', my_port_value.eval('(MKT_VALUE - INVESTMENT) *100 /INVESTMENT'))
    my_port_value['request_time'] = start_time
    my_port_value['request_time'] = pd.to_datetime(my_port_value['request_time'])
    return my_port_value
    

In [ ]:
def refresh_data(portfolio_df,daily_time_line):
    market_data = load_price_history(portfolio_df['SYMBOL'].unique(),days=1)
    port_data = pd.DataFrame({})
    for item in market_data:
        port_data = port_data.append(item.data,ignore_index=True)
    
    port_data = port_data.reset_index()[['SYMBOL','c','request_time']]
    
    if('pr_lastPrice' in portfolio_df):
        portfolio_df = portfolio_df.drop(['pr_lastPrice'],axis=1)
    if('request_time' in portfolio_df):
        portfolio_df = portfolio_df.drop(['request_time'],axis=1) 
        
    
    portfolio_df = portfolio_df.merge(port_data)
    portfolio_df = portfolio_df.rename(columns={'c':'pr_lastPrice'})
    portfolio_df['pr_pChange'] = portfolio_df.eval('(pr_lastPrice - pr_previousClose)*100/pr_previousClose ')
    portfolio_df = enrich_portfolio(portfolio_df)
    #portfolio_df = portfolio_df.merge(raw_port)
    # update portfolio
    if('load_time' not in daily_time_line or daily_time_line['load_time'].max() != portfolio_df['load_time'].max()):
        update_portfolio_file(portfolio_df,portfolio_name,False)
        daily_time_line = daily_time_line.append(portfolio_df[portfolio_df['QTY']>0])
        max_count = len(portfolio_df) * 60
        if(len(daily_time_line) > max_count):
            daily_time_line = daily_time_line.groupby(['SYMBOL','DATE']).head(10).append(daily_time_line.groupby(['SYMBOL','DATE']).tail(40))
    return portfolio_df,daily_time_line


In [ ]:
# time_interval = '30' #in minute 
def load_price_history(tickers,days=3650,time_interval="1D"):
    market_price_url ='https://priceapi.moneycontrol.com/techCharts/techChartController/history'
    market_base_url='https://www.moneycontrol.com/'
    start_time = datetime.today()
    market_data = []
    to_time = int(datetime.today().timestamp())
    from_time = int((datetime.today() - timedelta(days=days)).timestamp())
    # Create URL data
    for symbol in tickers:
        payload = {'symbol':symbol,'resolution':time_interval,'from': from_time,'to':to_time}
        market_data.append(MyURL(symbol,market_price_url,payload=payload,content_type='json',parent_page=market_base_url))
    # load market data
    for url in market_data:
        
        json = loadURL(url,True)
        try:
            del json['s']
            #display(json['h'][1:10])
            ticker_df = pd.DataFrame()
            for element in json:
                ticker_df[element] = json[element]

            ticker_df['time'] = ticker_df['t'].apply(lambda t: datetime.fromtimestamp(t)) 
            ticker_df['SYMBOL'] = url.name
            ticker_df['request_time'] = start_time
            ticker_df['request_time'] = pd.to_datetime(ticker_df['request_time'])
            url.data = ticker_df
        except:
            print('Error loading url ',url)
            
    print(len(market_data))        
    return market_data


    
    
    
    

In [ ]:
port_agg_col = {
            'INVESTMENT':"sum",
            'REAL': "sum",
            'MKT_VALUE':"sum",
            'CHANGE':"sum",
            'UNREAL': "sum",
        }

indus_agg_col = port_agg_col.copy()
indus_agg_col['SYMBOL'] = concat

overall_agg_col = port_agg_col.copy()
for key in overall_agg_col:
    overall_agg_col[key] = [overall_agg_col[key]]
    
def _color_red_or_green(val):
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

def display_intraday_change(daily_time_line):
    
    fig = go.Figure()
    for ticker in daily_time_line['SYMBOL'].unique():
        ticker_data = ticker_filter(daily_time_line,ticker)
        ticker_data = ticker_data.groupby(['request_time','SYMBOL']).agg(port_agg_col).reset_index()
        fig.add_trace(go.Scatter(x=ticker_data['request_time'], y=ticker_data['CHANGE'],
                            #mode='lines+markers',
                            mode='lines',
                            name=ticker))
    agg_data = daily_time_line.groupby('request_time').agg({'CHANGE':'sum','UNREAL':'sum'})
    start_bar = 2
    end_bar = 5
    if (len(agg_data) > (start_bar+end_bar)):
        agg_data = agg_data.head(start_bar).append(agg_data.tail(end_bar)).reset_index()
    else:
        agg_data = agg_data.reset_index()
    #agg_data['width'] = 1.5
    fig.add_trace(go.Bar(x=agg_data['request_time'],y=agg_data['CHANGE'],marker_color='indianred',name='--Today--'))
    
    #fig.update_xaxes(
    #    rangebreaks=[dict(values=daily_time_line['request_time'].unique())] # hide dates with no values
    #)
    #fig.add_trace(go.Bar(x=agg_data['request_time'],y=agg_data['UNREAL'],marker_color='darkblue',name='--OverAll--'))
    
    #fig.update_layout(
    #yaxis = dict(
    #        tickmode = 'array',
    #        tickvals = [-600,-300,-50,0,100, 300, 500, 1000, 3000, 5000, 10000]
    #    )
    #)
    
    fig.show()
    
def profit_info(sold_port_df):
    overall_agg = sold_port_df.agg({
            'INVESTMENT':["sum"],
            'REAL': ["sum"]})
    year_agg = sold_port_df.groupby('CATEGORY').agg({
            'INVESTMENT':"sum",
            'REAL': "sum"})
    display(year_agg)
    display(overall_agg)
    
def basic_port_info(portfolio_df):
    load_timing = portfolio_df['request_time'].agg(['min','max'])
    display(HTML('<b>Timing:</b> <I>'+ load_timing[0].strftime('%d %b %Y [ %H:%M:%S - ') + load_timing[1].strftime('%H:%M:%S ]') +'</I>'))
    
    overall_agg = portfolio_df.agg(overall_agg_col)
    year_agg = portfolio_df.groupby('CATEGORY').agg(port_agg_col)
    symbol_agg = portfolio_df.groupby('SYMBOL').agg(port_agg_col)
    
    industry_agg = portfolio_df[portfolio_df['QTY']>0].groupby('m_industry').agg(indus_agg_col)
    
    #viridis / inferno / plasma / magma
    cmap = 'viridis'
    #fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,5))
    #plt.subplots_adjust(bottom=.1)
    
    display_full(portfolio_df.set_index(['CATEGORY','SYMBOL']).sort_values(['CATEGORY','UNREAL %'],ascending=[False,True]))
    
    layout = {'height':400,'width':800,'hovermode': 'x'}
    industry_agg[["INVESTMENT","MKT_VALUE","REAL"]].iplot(kind='bar',layout=layout,subplots=False)
    
    symbol_agg[["INVESTMENT","MKT_VALUE","REAL"]].iplot(kind='bar',subplots=False,layout=layout)
    
    #for ax in axes:
        #plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    #plt.setp(ax2.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    #plt.xticks(rotation=30)
    #fig.delaxes(axes[1][0])
    #fig.delaxes(axes[1][1])
    #fig.tight_layout()
    #plt.show()
    
             #.style.applymap(_color_red_or_green, subset=['UNREAL','CHANGE'])
    display(industry_agg)
    display(year_agg)
    display(overall_agg)